In [3]:
%load_ext autoreload
%autoreload 2

In [28]:
from sagas import tool
tool.init_logger()

In [29]:
tool.color_print('red', 'hello')

hello


In [10]:
from unicodedata import normalize
word='、？'
word = normalize('NFKD', word).encode('ascii', 'ignore')
word

b'?'

In [12]:
import unicodedata
norm=lambda t: unicodedata.normalize('NFKC', t).encode('ascii', 'ignore').decode("utf-8")
norm('、？')

'?'

In [22]:
import sagas.tool as sag
import saai.tool as saa
saa.ner_jieba('我是中国的学生')

[{'start': 2, 'end': 4, 'value': '中国', 'entity': 'LOC', 'confidence': None}]

In [25]:
saa.ner_jieba_vis('我是中国的学生')

BoxMarkup('我是中国的学生', [Span(2, 4, 'LOC')])

In [26]:
from sagas.conf import resource_path
resource_path('sagas_conf.json')

'/Users/xiaofeiwu/jcloud/assets/langs/workspace/rasa/stack/sagas/conf/sagas_conf.json'

In [27]:
resource_path('logger.yml')

'/Users/xiaofeiwu/jcloud/assets/langs/workspace/rasa/stack/sagas/conf/logger.yml'

In [32]:
from sagas.conf import resource_dir
from fnmatch import fnmatch
resource_dir(lambda f: fnmatch(f, 'mod_*.json'))

['mod_multilang.json']

In [33]:
import os
os.path.exists('/tmp')

True

In [59]:
from sagas.nlu.ruleset_procs import cached_chunks, get_main_domains
get_main_domains('Ich stimme dir in diesem Punkt nicht zu.', 'de', 'corenlp')

('verb_domains',
 [{'index': '2',
   'text': 'stimme',
   'lemma': 'stimmen',
   'upos': 'VERB',
   'xpos': 'VVFIN',
   'feats': 'Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin',
   'governor': 0,
   'dependency_relation': 'root',
   'obl': [{'index': '6',
     'text': 'Punkt',
     'lemma': 'Punkt',
     'upos': 'NOUN',
     'xpos': 'NN',
     'feats': 'Case=Dat|Gender=Masc|Number=Sing',
     'governor': 2,
     'dependency_relation': 'obl',
     'case': [{'index': '4',
       'text': 'in',
       'lemma': 'in',
       'upos': 'ADP',
       'xpos': 'APPR',
       'feats': '_',
       'governor': 6,
       'dependency_relation': 'case'}],
     'det': [{'index': '5',
       'text': 'diesem',
       'lemma': 'dies',
       'upos': 'PRON',
       'xpos': 'PDAT',
       'feats': 'Case=Dat|Gender=Masc|Number=Sing|PronType=Dem',
       'governor': 6,
       'dependency_relation': 'det'}]}],
   'compound:prt': [{'index': '8',
     'text': 'zu',
     'lemma': 'zu',
     'upos': 'ADP',
 

In [79]:
from sagas.nlu.ruleset_procs import cached_chunks, get_main_domains
get_main_domains('彼のパソコンは便利じゃない。', 'ja', 'knp')

('predicts',
 [{'type': 'predicate',
   'lemma': '便利だ',
   'index': 2,
   'phonetic': 'べんりだ',
   'word': '便利じゃない。',
   'rel': 'D',
   'governor': 0,
   'domains': [['ガ', 1, 'パソコンは', 'パソコン', ['パソコン'], ['c_noun', 'x_n']]],
   'stems': []}])

In [76]:
from sagas.conf.conf import cf
from sagas.tracker_jupyter import enable_jupyter_tracker
from sagas.nlu.rules_header import *
# from sagas.nlu.inspectors import InspectorFixture
from sagas.nlu.patterns import print_result
from sagas.nlu.uni_remote import dep_parse
from sagas.nlu.corenlp_parser import get_chunks
from sagas.nlu.rules_meta import build_meta

from pprint import pprint

enable_jupyter_tracker()
cf.enable_opt('print_not_matched')

def dynamic_rule(data, rule_str, name='_none_', engine=None):
    # ft=InspectorFixture()
    # domains, meta=ft.request_domains(data, engine=engine)
    if engine is None:
        engine=cf.engine(data['lang'])
    pipelines=['predicts']
    doc_jsonify, resp = dep_parse(data['sents'], data['lang'], engine, pipelines)
    if doc_jsonify is not None:
        if len(resp['predicts'])>0:
            domains_set=resp['predicts']
        else:
            domains_set=get_chunks(doc_jsonify)
        
        for r in domains_set:
            domains = r['domains']            
            meta = build_meta(r, data)
            print(r['type'], meta['word'], meta['lemma'], list(meta.keys()))
            pprint(domains)
            agency = ['c_pron', 'c_noun']
            rs=eval(f"[Patterns(domains, meta, 5, name='{name}').{rule_str}]")
            print_result(rs)

data = {'lang': 'de', "sents": 'Ich stimme dir in diesem Punkt nicht zu.'}
dynamic_rule(data, """verb(nsubj=agency, obj=kindof('print_media/artifact', 'n'))""")

verb_domains stimme stimmen ['rel', 'lemma', 'word', 'stems', 'lang', 'sents']
[('nsubj', '1', 'Ich', 'ich', ['Ich'], ['c_pron', 'x_pper']),
 ('obj', '3', 'dir', 'dur', ['dir'], ['c_pron', 'x_prf']),
 ('obl', '6', 'Punkt', 'Punkt', ['in', 'diesem', 'Punkt'], ['c_noun', 'x_nn']),
 ('advmod', '7', 'nicht', 'nicht', ['nicht'], ['c_part', 'x_ptkneg']),
 ('compound:prt', '8', 'zu', 'zu', ['zu'], ['c_adp', 'x_ptkvz']),
 ('punct', '9', '.', '.', ['.'], ['c_punct', 'x_$.'])]
✖ (_none_) verb with nsubj is ['c_pron', 'c_noun']: True, obj is kind_of(print_media/artifact,n): False


In [65]:
data = {'lang': 'en', "sents": 'They read these newspapers.'}
dynamic_rule(data, """verb(nsubj=agency, obj=kindof('print_media/artifact', 'n'))""")

[('nsubj', '1', 'They', 'they', ['They'], ['c_pron', 'x_prp']),
 ('obj',
  '4',
  'newspapers',
  'newspaper',
  ['these', 'newspapers'],
  ['c_noun', 'x_nns']),
 ('punct', '5', '.', '.', ['.'], ['c_punct', 'x_.'])]
✔ (_none_) verb with nsubj is ['c_pron', 'c_noun']: True, obj is kind_of(print_media/artifact,n): True
.. results 1
{'kind_of/default/obj'}


{'kind_of/default/...': {'category': 'print_media/artifact', 'pos': 'n'}}

In [77]:
data = {'lang': 'en', "sents": 'I am a student.'}
dynamic_rule(data, """verb(nsubj=agency, obj=kindof('print_media/artifact', 'n'))""")

aux_domains am be ['pos', 'head', 'lemma', 'word', 'stems', 'lang', 'sents']
[('nsubj', '1', 'I', 'I', ['I'], ['c_pron', 'x_prp']),
 ('cop', '2', 'am', 'be', ['am'], ['c_aux', 'x_vbp']),
 ('det', '3', 'a', 'a', ['a'], ['c_det', 'x_dt']),
 ('punct', '5', '.', '.', ['.'], ['c_punct', 'x_.'])]
✖ (_none_) verb with nsubj is ['c_pron', 'c_noun']: True, obj is kind_of(print_media/artifact,n): False


In [68]:
data = {'lang': 'ja', "sents": 'このお土産はきれいで安いです。'}
# data = {'lang': 'ja', "sents": 'このお土産は安いです。'}
dynamic_rule(data, """verb(behaveof('cheap', 'a'), ガ=kindof('present', 'n'))""", 
             engine='knp')

[]
✖ (_none_) verb with pos is behave_of(cheap,a): False, ガ is kind_of(present,n): False
[['ガ', 1, 'お土産は', '土産', ['御', '土産'], ['c_x', 'x_p']],
 ['修飾', 2, 'きれいで', 'きれいで', ['綺麗だ'], ['c_adj', 'x_j']]]
✔ (_none_) verb with pos is behave_of(cheap,a): True, ガ is kind_of(present,n): True
.. results 2
{'behave_of/default/predicate', 'kind_of/default/ガ'}


{'behave_of/default/...': {'category': 'cheap', 'pos': 'a'},
 'kind_of/default/...': {'category': 'present', 'pos': 'n'}}

In [69]:
data = {'lang': 'ja', "sents": '彼のパソコンは便利じゃない。'}
dynamic_rule(data, """subj('adj',ガ=kindof('artifact', 'n'))""", 
             engine='knp')

[['ガ', 1, 'パソコンは', 'パソコン', ['パソコン'], ['c_noun', 'x_n']]]
✖ (_none_) subj with pos is ('adj',): False, ガ is kind_of(artifact,n): True


In [2]:
eval(compile('[42]', '<string>', 'eval')) 

[42]